In [2]:
import tkinter as tk
from tkinter import ttk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from scipy.signal import square, sawtooth, find_peaks

# Parámetros globales
fs = 2048                 # Frecuencia de muestreo (Hz)
duration = 1.0            # Duración de la señal (segundos)
t = np.linspace(0, duration, int(fs * duration), endpoint=False)  # Vector de tiempo

def generar_senal():
    # Lectura de valores desde las entradas
    amplitudes = [float(a.get() or 0) for a in amp_entries]
    frecuencias = [float(f.get() or 0) for f in freq_entries]
    tipos = [tipo.get() for tipo in type_selectors]
    duty_cycles = [duty.get() for duty in duty_sliders]

    # Inicializa señal en cero
    signal = np.zeros_like(t)

    # Construcción de la señal sumando las componentes activas
    for A, f, tipo, duty in zip(amplitudes, frecuencias, tipos, duty_cycles):
        if A > 0 and f > 0:
            if tipo == "Senoidal":
                signal += A * np.sin(2 * np.pi * f * t)
            elif tipo == "Cuadrada":
                signal += A * square(2 * np.pi * f * t, duty=duty/100)
            elif tipo == "Triangular":
                signal += A * sawtooth(2 * np.pi * f * t, width=0.5)

    # --- Análisis FFT con ventana Hanning o Blackman ---
    N = len(signal)
    window = np.hanning(N)         #Ventana Hanning
    #window = np.blackman(N)       #Ventana Blackman
    signal_windowed = signal * window
    fft_vals = np.fft.fft(signal_windowed)
    freqs = np.fft.fftfreq(N, 1/fs)
    idx = np.where(freqs >= 0)
    freqs = freqs[idx]
    amps = (2 * np.abs(fft_vals[idx])) / N
    amps = amps / np.mean(window)   # Corrección por la ventana

    # --- Detección de picos en el espectro ---
    peak_indices, _ = find_peaks(amps, height=np.max(amps)*0.05)  # picos >5% del máx
    peak_freqs = freqs[peak_indices]
    peak_amps = amps[peak_indices]

    # --- Cálculo de RMS ---
    rms_val = np.sqrt(np.mean(signal**2))

    # --- Cálculo de THD (Total Harmonic Distortion) ---
    f_nonzero = [f for f in frecuencias if f > 0]
    if f_nonzero:
        f1 = min(f_nonzero)  # frecuencia fundamental
        harmonics = []
        for n in range(1, 10):
            f_target = n * f1
            idx_h = np.argmin(np.abs(freqs - f_target))
            harmonics.append(amps[idx_h])
        fundamental_amp = harmonics[0]
        other = harmonics[1:]
        thd = np.sqrt(np.sum(np.array(other)**2)) / (fundamental_amp + 1e-12)
    else:
        thd = 0.0

    # --- Limpieza de gráficos anteriores ---
    for widget in frame_signal.winfo_children(): widget.destroy()
    for widget in frame_fft.winfo_children(): widget.destroy()
    for widget in frame_info.winfo_children(): widget.destroy()

    # --- Gráfico de la señal en el tiempo ---
    fig1, ax1 = plt.subplots(figsize=(5, 2))
    ax1.plot(t, signal)
    ax1.set_title("Señal generada")
    ax1.set_xlabel("Tiempo [s]")
    ax1.set_ylabel("Amplitud [V]")
    ax1.grid(True)
    ax1.set_xlim(0, 0.1)  # se muestra solo una porción
    canvas1 = FigureCanvasTkAgg(fig1, master=frame_signal)
    canvas1.draw()
    canvas1.get_tk_widget().pack(expand=True, fill='both')

    # --- Gráfico del espectro de frecuencias ---
    fig2, ax2 = plt.subplots(figsize=(5, 2))
    ax2.plot(freqs, amps)
    ax2.set_title("Espectro")
    ax2.set_xlabel("Frecuencia [Hz]")
    ax2.set_ylabel("Amplitud [V]")
    ax2.set_xlim(0, freqs.max() * 1.05)
    ax2.grid(True)

    # --- Anotación interactiva sobre los picos ---
    annot = ax2.annotate("", xy=(0,0), xytext=(15,15), textcoords="offset points",
                         bbox=dict(boxstyle="round", fc="yellow", alpha=0.8),
                         arrowprops=dict(arrowstyle="->"))
    annot.set_visible(False)

    # Actualiza la anotación al mover el cursor
    def update_annot(event):
        if event.inaxes == ax2 and event.xdata is not None:
            distances = np.abs(peak_freqs - event.xdata)
            idx_min = np.argmin(distances)
            if distances[idx_min] < 5:  # distancia máxima para mostrar
                x, y = peak_freqs[idx_min], peak_amps[idx_min]

                # Ajuste de posición de la etiqueta según bordes
                offset_x, offset_y = 15, 15
                if x > ax2.get_xlim()[1] * 0.8: offset_x = -60
                if y > ax2.get_ylim()[1] * 0.8: offset_y = -40

                annot.xy = (x, y)
                annot.set_position((offset_x, offset_y))
                annot.set_text(f"f: {x:.2f} Hz\nA: {y:.4f} V")
                annot.set_visible(True)
                fig2.canvas.draw_idle()
            else:
                annot.set_visible(False)
                fig2.canvas.draw_idle()
        else:
            annot.set_visible(False)
            fig2.canvas.draw_idle()

    # Inserta el gráfico en el frame
    canvas2 = FigureCanvasTkAgg(fig2, master=frame_fft)
    canvas2.draw()
    widget_canvas = canvas2.get_tk_widget()
    widget_canvas.pack(expand=True, fill='both')

    # Conecta el evento del mouse
    fig2.canvas.mpl_connect("motion_notify_event", update_annot)

    # --- Muestra valores numéricos ---
    tk.Label(frame_info, text=f"RMS: {rms_val:.4f} V").pack()
    tk.Label(frame_info, text=f"THD: {thd*100:.2f}%").pack()

# --- Construcción de la interfaz gráfica principal ---
root = tk.Tk()
root.title("PES - Etapa 1 - Fortunati / Martinez")

# Configuración de filas y columnas expandibles
for i in range(7):
    root.grid_columnconfigure(i, weight=1)
root.grid_rowconfigure(4, weight=1)
root.grid_rowconfigure(5, weight=1)
root.grid_rowconfigure(6, weight=1)

amp_entries = []
freq_entries = []
type_selectors = []
duty_sliders = []

# --- Encabezados ---
tk.Label(root, text="AMPLITUD").grid(row=0, column=0, padx=5, pady=5)
tk.Label(root, text="FRECUENCIA").grid(row=1, column=0, padx=5, pady=5)
tk.Label(root, text="TIPO").grid(row=2, column=0, padx=5, pady=5)
tk.Label(root, text="DUTY (%)").grid(row=3, column=0, padx=5, pady=5)

# --- Entradas de parámetros para 4 señales ---
for i in range(4):
    a_entry = tk.Entry(root, width=6)
    f_entry = tk.Entry(root, width=6)
    a_entry.grid(row=0, column=i+1, padx=5)
    f_entry.grid(row=1, column=i+1, padx=5)
    amp_entries.append(a_entry)
    freq_entries.append(f_entry)

    tipo_cb = ttk.Combobox(root, values=["Senoidal", "Cuadrada", "Triangular"], width=8)
    tipo_cb.current(0)
    tipo_cb.grid(row=2, column=i+1, padx=5)
    type_selectors.append(tipo_cb)

    duty_var = tk.IntVar(value=50)
    duty_slider = tk.Scale(root, from_=10, to=50, orient=tk.HORIZONTAL, resolution=5, variable=duty_var)
    duty_slider.grid(row=3, column=i+1, padx=5)
    duty_sliders.append(duty_var)

# --- Botón para generar la señal ---
btn = tk.Button(root, text="GENERAR", command=generar_senal)
btn.grid(row=0, column=6, rowspan=4, padx=10)

# --- Frames donde se colocan los gráficos y resultados ---
frame_signal = tk.Frame(root)
frame_signal.grid(row=4, column=0, columnspan=7, pady=10, sticky="nsew")

frame_fft = tk.Frame(root)
frame_fft.grid(row=5, column=0, columnspan=7, pady=10, sticky="nsew")

frame_info = tk.Frame(root)
frame_info.grid(row=6, column=0, columnspan=7, pady=10, sticky="nsew")

# Inicia el bucle principal de la interfaz
root.mainloop()
